Binarization is the process of transforming data features of any entity into vectors of binary numbers to make classifier algorithms more efficient.

### import libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split,cross_val_score

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

from sklearn.preprocessing import Binarizer

from sklearn.compose import ColumnTransformer

### load dataset

In [2]:
df = pd.read_csv("data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df = df.loc[:, ["Age", "Fare", "SibSp", "Parch", "Survived"]]
df.head()

,Age,Fare,SibSp,Parch,Survived
0,22.0,7.2500,1,0,0
1,38.0,71.2833,1,0,1
2,26.0,7.9250,0,0,1
3,35.0,53.1000,1,0,1
4,35.0,8.0500,0,0,0


In [4]:
df.dropna(inplace=True)

In [5]:
df["family"] = df["SibSp"]+df["Parch"]

In [6]:
df.drop(["SibSp", "Parch"], axis=1, inplace=True)
df.head()

,Age,Fare,Survived,family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


### splitting data into train and test set

In [7]:
X = df.drop("Survived", axis=1)
y = df["Survived"]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape

((571, 3), (143, 3))

### without using binarization

In [9]:
clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.6293706293706294

In [10]:
# using cross val
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

0.6485524256651017

### using binarization

In [11]:
trf = ColumnTransformer([
    ('bin',Binarizer(copy=False),['family'])
],remainder='passthrough')

In [12]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [13]:
pd.DataFrame(X_train_trf, columns=['family','Age','Fare'])

,family,Age,Fare
0,0.0,28.0,10.5000
1,1.0,52.0,79.6500
2,1.0,22.0,7.2500
3,1.0,27.0,11.1333
4,0.0,19.0,14.5000
...,...,...,...
566,0.0,36.0,10.5000
567,1.0,26.0,26.0000
568,1.0,20.0,9.8250
569,1.0,25.0,26.0000


In [14]:
clf = DecisionTreeClassifier()

clf.fit(X_train_trf,y_train)

y_pred2 = clf.predict(X_test_trf)

accuracy_score(y_test,y_pred2)

0.6433566433566433

In [15]:
#using cross validation
X_trf = trf.fit_transform(X)

np.mean(cross_val_score(DecisionTreeClassifier(), X_trf, y, cv=10, scoring='accuracy'))

0.6219679186228483